# Wrangling of Video Game Review Information from Steam Website
In this initial step I'll extract video game information and corresponding user reviews through Valve's Steam digital distribution company and supporting websites.

The data is described at https://partner.steamgames.com/doc/store/getreviews and https://www.steamspy.com/about, and extracted using the steamreviews API from https://pypi.org/project/steamreviews/ and code from Nik Davis at https://nik-davis.github.io/posts/2019/steam-data-collection/.

In [9]:
# standard library imports
import json
import os.path
import time
import datetime as dt
import re
import csv
import statistics

# third-party imports
import pandas as pd
import numpy as np
import requests
import steamreviews

In [10]:
# set your personal folder
path_steam = "C:/users/ggibs/steam_game_recommender"

In [11]:
# URLs and subfolders
steam_url = "http://store.steampowered.com/api/appdetails/"
steamspy_url = "https://steamspy.com/api.php"

path_raw = os.path.join(path_steam, "data/raw/")
path_interim = os.path.join(path_steam, "data/interim/")
path_external = os.path.join(path_steam, "data/external/")

## Define Functions

In [12]:
def get_request(url, parameters=None):
    """Return json-formatted response of a get request using optional parameters.
    
    Parameters
    ----------
    url : string
    parameters : {'parameter': 'value'}
        parameters to pass as part of get request
    
    Returns
    -------
    json_data
        json-formatted response (dict-like)
    """
    try:
        response = requests.get(url=url, params=parameters)
    except SSLError as s:
        print('SSL Error:', s)
        
        for i in range(5, 0, -1):
            print('\rWaiting... ({})'.format(i), end='')
            time.sleep(1)
        print('\rRetrying.' + ' '*10)
        
        # recursively try again
        return get_request(url, parameters)
    
    if response:
        return response.json()
    else:
        # response is none usually means too many requests. Wait and try again 
        print('No response, waiting 10 seconds...')
        time.sleep(10)
        print('Retrying.')
        return get_request(url, parameters)

In [13]:
# index to track records completed

def reset_index(download_path, index_filename):
    """Reset index in file to 0."""
    rel_path = os.path.join(download_path, index_filename)
    
    with open(rel_path, 'w') as f:
        print(0, file=f)
        

def get_index(download_path, index_filename):
    """Retrieve index from file, returning 0 if file not found."""
    try:
        rel_path = os.path.join(download_path, index_filename)

        with open(rel_path, 'r') as f:
            index = int(f.readline())
    
    except FileNotFoundError:
        index = 0
        
    return index

In [14]:
def prepare_data_file(download_path, filename, index, columns):
    """Create file and write headers if index is 0."""
    if index == 0:
        rel_path = os.path.join(download_path, filename)

        with open(rel_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=columns)
            writer.writeheader()

In [7]:
def parse_steam_request(appid, name):
    """Unique parser to handle data from Steam Store API.
    
    Returns : json formatted data (dict-like)
    """
    url = steam_url
    parameters = {"appids": appid}
    
    json_data = get_request(url, parameters=parameters)
    json_app_data = json_data[str(appid)]
    
    if json_app_data['success']:
        data = json_app_data['data']
    else:
        data = {'name': name, 'steam_appid': appid}
        
    return data

In [38]:
def parse_steamspy_request(appid, name):
    """Parser to handle SteamSpy API data."""
    url = steamspy_url
    parameters = {"request": "appdetails", "appid": appid}
    
    json_data = get_request(url, parameters)
    return json_data

In [8]:
def get_app_data(start, stop, parser, pause):
    """Return list of app data generated from parser.
    
    parser : function to handle request
    """
    app_data = []
    
    # iterate through each row of app_list, confined by start and stop
    for index, row in app_list[start:stop].iterrows():
        print('Current index: {}'.format(index), end='\r')
        
        appid = row['appid']
        name = row['name']

        # retrive app data for a row, handled by supplied parser, and append to list
        data = parser(appid, name)
        app_data.append(data)

        time.sleep(pause) # prevent overloading api with requests
    
    return app_data

In [32]:
def process_batches(parser, app_list, download_path, data_filename, index_filename,
                    columns, begin=0, end=-1, batchsize=100, pause=1):
    """Process app data in batches, writing directly to file.
    
    parser : custom function to format request
    app_list : dataframe of appid and name
    download_path : path to store data
    data_filename : filename to save app data
    index_filename : filename to store highest index written
    columns : column names for file
    
    Keyword arguments:
    
    begin : starting index (get from index_filename, default 0)
    end : index to finish (defaults to end of app_list)
    batchsize : number of apps to write in each batch (default 100)
    pause : time to wait after each api request (defualt 1)
    
    returns: none
    """
    print('Starting at index {}:\n'.format(begin))
    
    # by default, process all apps in app_list
    if end == -1:
        end = len(app_list) + 1
    
    # generate array of batch begin and end points
    batches = np.arange(begin, end, batchsize)
    batches = np.append(batches, end)
    
    apps_written = 0
    batch_times = []
    
    for i in range(len(batches) - 1):
        start_time = time.time()
        
        start = batches[i]
        stop = batches[i+1]
        
        app_data = get_app_data(start, stop, parser, pause)
        
        rel_path = os.path.join(download_path, data_filename)
        
        # writing app data to file
        with open(rel_path, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=columns, extrasaction='ignore')
            
            for j in range(3,0,-1):
                print("\rAbout to write data, don't stop script! ({})".format(j), end='')
                time.sleep(0.5)
            
            writer.writerows(app_data)
            print('\rExported lines {}-{} to {}.'.format(start, stop-1, data_filename), end=' ')
            
        apps_written += len(app_data)
        
        idx_path = os.path.join(download_path, index_filename)
        
        # writing last index to file
        with open(idx_path, 'w') as f:
            index = stop
            print(index, file=f)
            
        # logging time taken
        end_time = time.time()
        time_taken = end_time - start_time
        
        batch_times.append(time_taken)
        mean_time = statistics.mean(batch_times)
        
        est_remaining = (len(batches) - i - 2) * mean_time
        
        remaining_td = dt.timedelta(seconds=round(est_remaining))
        time_td = dt.timedelta(seconds=round(time_taken))
        mean_td = dt.timedelta(seconds=round(mean_time))
        
        print('Batch {} time: {} (avg: {}, remaining: {})'.format(i, time_td, mean_td, remaining_td))
            
    print('\nProcessing batches complete. {} apps written'.format(apps_written))

## Game List and Game Information from Steam and SteamSpy
### Columns can duplicate between sites

In [ ]:
# produce list of games
url = steamspy_url
parameters = {"request": "all"}

# request 'all' from steam spy and parse into dataframe
json_data = get_request(url, parameters=parameters)
steam_spy_all = pd.DataFrame.from_dict(json_data, orient='index')

# generate sorted app_list from steamspy data
app_list = steam_spy_all[['appid', 'name']].sort_values('appid').reset_index(drop=True)

# export, comment out after first run
app_list.to_csv(os.path.join(path_external, app_list.csv), index=False)

In [10]:
# read from stored csv
app_list = pd.read_csv(os.path.join(path_external, "app_list.csv"))

In [ ]:
# set files and columns
download_path = path_external
steamspy_data = 'steamspy_data.csv'
steamspy_index = 'steamspy_index.txt'

steamspy_columns = [
    'appid', 'name', 'developer', 'publisher', 'score_rank', 'positive',
    'negative', 'userscore', 'owners', 'average_forever', 'average_2weeks',
    'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
    'languages', 'genre', 'ccu', 'tags'
]

reset_index(download_path, steamspy_index)
index = get_index(download_path, steamspy_index)

# Wipe data file if index is 0
prepare_data_file(download_path, steamspy_data, index, steamspy_columns)

process_batches(
    parser=parse_steamspy_request,
    app_list=app_list,
    download_path=download_path, 
    data_filename=steamspy_data,
    index_filename=steamspy_index,
    columns=steamspy_columns,
    begin=index,
#    end=20,
#    batchsize=5,
#    pause=1
)

### Sample output

Starting at index 0:

Exported lines 0-99 to steamspy_data.csv. Batch 0 time: 0:02:16 (avg: 0:02:16, remaining: 0:22:37)  
Exported lines 100-199 to steamspy_data.csv. Batch 1 time: 0:02:14 (avg: 0:02:15, remaining: 0:20:12)  
Exported lines 200-299 to steamspy_data.csv. Batch 2 time: 0:02:20 (avg: 0:02:17, remaining: 0:18:12)  
Exported lines 300-399 to steamspy_data.csv. Batch 3 time: 0:02:21 (avg: 0:02:18, remaining: 0:16:03)  
Exported lines 400-499 to steamspy_data.csv. Batch 4 time: 0:02:38 (avg: 0:02:22, remaining: 0:14:09)  
Exported lines 500-599 to steamspy_data.csv. Batch 5 time: 0:02:16 (avg: 0:02:21, remaining: 0:11:43)  
Exported lines 600-699 to steamspy_data.csv. Batch 6 time: 0:02:22 (avg: 0:02:21, remaining: 0:09:23)  
Exported lines 700-799 to steamspy_data.csv. Batch 7 time: 0:02:23 (avg: 0:02:21, remaining: 0:07:03)  
Exported lines 800-899 to steamspy_data.csv. Batch 8 time: 0:02:14 (avg: 0:02:20, remaining: 0:04:41)  
Exported lines 900-999 to steamspy_data.csv. Batch 9 time: 0:02:14 (avg: 0:02:20, remaining: 0:02:20)  
Exported lines 1000-1000 to steamspy_data.csv. Batch 10 time: 0:00:02 (avg: 0:02:07, remaining: 0:00:00)  

Processing batches complete. 1000 apps written

In [ ]:
# Set file parameters
download_path = path_external
steam_app_data = 'steam_app_data.csv'
steam_index = 'steam_index.txt'

steam_columns = [
    'type', 'name', 'steam_appid', 'required_age', 'is_free', 'controller_support',
    'dlc', 'detailed_description', 'about_the_game', 'short_description', 'fullgame',
    'supported_languages', 'header_image', 'website', 'pc_requirements', 'mac_requirements',
    'linux_requirements', 'legal_notice', 'drm_notice', 'ext_user_account_notice',
    'developers', 'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
    'platforms', 'metacritic', 'reviews', 'categories', 'genres', 'screenshots',
    'movies', 'recommendations', 'achievements', 'release_date', 'support_info',
    'background', 'content_descriptors'
]

# Store highest index so can continue across sessions)
reset_index(download_path, steam_index)

# Retrieve last index downloaded from file
index = get_index(download_path, steam_index)

# Wipe or create data file and write headers if index is 0
prepare_data_file(download_path, steam_app_data, index, steam_columns)

# Set batch parameters, remove batchsize and end to run through entire app list
process_batches(
    parser=parse_steam_request,
    app_list=app_list,
    download_path=download_path,
    data_filename=steam_app_data,
    index_filename=steam_index,
    columns=steam_columns,
    begin=index,
#    end=30,
#    batchsize=10
)

## Game Reviews

In [ ]:
# constrain API query of reviews
# reference: https://partner.steamgames.com/doc/store/getreviews
request_params = dict()
request_params['language'] = 'english'
request_params['purchase_type'] = 'steam'  # customer paid for game
request_params['day_range'] = '1095'  # the past three years

In [ ]:
# convert dataframe to list
list_of_games_id = app_list.appid.to_list()

In [ ]:
# 150 query limit, then 5 min break, took 19 hours to complete
os.chdir(path_raw)
steamreviews.download_reviews_for_app_id_batch(list_of_games_id, chosen_request_params=request_params)

In [5]:
review_columns = [
    'recommendationid', 'language', 'review', 'timestamp_created', 'timestamp_updated', 'voted_up', 'votes_up', 'votes_funny',
    'weighted_vote_score', 'comment_count', 'steam_purchase', 'received_for_free', 'written_during_early_access', 
    'author.steamid', 'author.num_games_owned', 'author.num_reviews', 'author.playtime_forever', 'author.playtime_last_two_weeks',
    'author.playtime_at_review', 'author.last_played', 'timestamp_dev_responded', 'developer_response', 'appid'
]
review_df = pd.DataFrame(columns=review_columns)

In [ ]:
# for each game downloaded, open review json, flatten nested column Author, and append to dataframe
for file in os.listdir():
    print(time.strftime("%H:%M:%S", time.localtime()))
    f = open(file)
    game_num = re.findall("review_(\d+).json", file)
    print(game_num)
    review_dict = json.load(f)
    if review_dict['reviews']:    
        for key in review_dict['reviews']:
            df = pd.json_normalize(review_dict['reviews'][key])
            df['appid'] = game_num
            review_df = review_df.append(df)
    f.close()

### Sample output with each game ID
12:37:18  
['599140']  
12:38:06  
['601150']  
12:39:53  
['601510']  
12:39:53  
['611500']  

In [ ]:
# Application ID and Recommendation ID are the keys
review_df.set_index(['appid', 'recommendationid'], inplace=True)

In [ ]:
# output dataframe to csv
reviews_df.to_csv(os.path.join(path_interim ,"reviews.csv")